In [1]:
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
# Downloading tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-26 10:10:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-07-26 10:10:15 (27.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# This block identifies all unique character in string or tet stored in variable 'text'
chars = sorted(list(set(text))) # Extracts unique characters from the string text, converts the set to a list, and sorts it alphabetically.
vocab_size = len(chars)
print(''.join(chars)) # prints sorted list
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from characters to integers + defines function for encoding and decoding strings
stoi = { ch:i for i,ch in enumerate(chars) } # create a dictionary mapping characters to integers // i is index, ch is character in enumerate (this is fashion of enumerate)
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the first 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size] # x represents the input sequence // takes the first block_size number of elements
y = train_data[1:block_size+1] # takes elements from 2nd elements up to block_size+1 number of elements // is essentially x but shifted one position to the right
                               # This is used as the target sequence for training. Where each element in y is the sequence that model should predict after seeing sequence in 'x'

for t in range(block_size): # This for loop iterates over each position in the block_size, effectively allowing you to demonstrate or train the model on increasing amounts of context.
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

# note: as seen below, when provided a sequence in training set. The model actually tries to train the data batch as just [1] to [9], to predict the
#       token. So that the model are able to capture semantic meaning better, and also is able to deal with small inputs

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


## Word Notes

Query: What is byte-pair encoding?

Byte-Pair Encoding (BPE) is a data compression technique originally designed to efficiently compress text data. It has been adapted in natural language processing (NLP) to perform subword tokenization, which involves breaking down words into more manageable units (subwords) for models to process. This technique is especially useful in handling languages with rich morphology or in managing the vocabulary size in neural network models.

### How Byte-Pair Encoding Works in NLP

1. **Initialization**: Start with a vocabulary of individual characters or other base units (like bytes in its original application) from the training data.

2. **Frequency Count**: Count the frequency of pairs of adjacent symbols or characters in the text data.

3. **Merge Operations**: Identify the most frequently occurring pair of symbols and merge them into a single new symbol. This merged symbol is then added to the vocabulary.

4. **Iterative Process**: Repeat the merging process for a predetermined number of steps or until some criterion is met (like a desired vocabulary size). Each iteration merges the next most frequent pair.

5. **Tokenization**: Once the vocabulary of symbols (including merged symbols) is created, tokenize the text by greedily replacing the longest possible sequences in the text with the corresponding symbols in the vocabulary.

### Advantages of BPE in NLP

- **Efficiency in Unknown Words**: BPE allows a model to handle words not seen during training, by breaking them into known subwords. This is particularly useful in languages with compounding or agglutination where new word forms are frequent.

- **Balance Between Character and Word Level**: BPE provides a balance between character-level and word-level representations. It avoids the sparsity of character-level processing and the vocabulary explosion of word-level processing.

- **Contextual Awareness**: Subwords provide more contextual information than individual characters, helping models understand suffixes, prefixes, and roots, which are crucial for semantic and syntactic understanding.

### Usage in Modern NLP

BPE has been employed in several state-of-the-art NLP models and frameworks, notably in models like GPT (from OpenAI), BERT, and other Transformer-based architectures. These models use BPE or its variants (like SentencePiece or WordPiece) to preprocess text into subwords, which are then used for tasks like translation, text summarization, and question-answering.

Overall, BPE is a pivotal component in the preprocessing pipeline for many NLP tasks, helping to improve both the efficiency and effectiveness of models in handling diverse and complex language data.

The function `torch.stack` in PyTorch is used to concatenate a sequence of tensors along a new dimension. This is different from functions like `torch.cat`, which concatenate tensors along an existing dimension. `torch.stack` effectively combines tensors in such a way that the resulting tensor has an additional dimension compared to the input tensors.

### Usage of `torch.stack`

**Syntax**:
```python
torch.stack(tensors, dim=0)
```

- **tensors**: a sequence of tensors to concatenate. All tensors must have the same shape.
- **dim**: the dimension along which to concatenate the tensors. This dimension is added to the resulting tensor, and each tensor in the sequence will be sliced along this new dimension.

### Example

Let's consider you have three 2x2 tensors:

```python
import torch

# Define three 2x2 tensors
tensor1 = torch.tensor([[1, 2], [3, 4]])
tensor2 = torch.tensor([[5, 6], [7, 8]])
tensor3 = torch.tensor([[9, 10], [11, 12]])

# Stack these tensors along a new dimension
result = torch.stack([tensor1, tensor2, tensor3], dim=0)
print(result)
```

**Output**:
```
tensor([[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]],

        [[ 9, 10],
         [11, 12]]])
```

In this example:
- The original tensors are 2x2 matrices.
- The `torch.stack` function stacks them along a new first dimension (`dim=0`), resulting in a 3x2x2 tensor. This means we now have 3 matrices of size 2x2 stacked on top of each other.

### Practical Uses in Machine Learning

`torch.stack` is often used in machine learning and deep learning when you need to batch together multiple tensors of the same size, such as when combining individual feature tensors into a batch tensor that can be passed to a model. This operation is critical for efficient computation, allowing batch processing of data rather than processing one item at a time.

The syntax `(batch_size,)` in Python is used to create a tuple containing a single element, which in this context is the `batch_size`. This notation is necessary because simply writing `batch_size` without the comma and parentheses would be interpreted as an integer, not a tuple. In Python, the trailing comma is what distinguishes a tuple from a regular parenthesis-wrapped expression.

### Context in PyTorch:

When using PyTorch, especially in functions like `torch.randint`, specifying dimensions as a tuple is common. For example, if you want to generate random integers within a certain range and shape the output as a tensor with dimensions defined by that tuple, you'd use:

```python
torch.randint(high, (shape_tuple))
```

- **high**: The upper boundary (exclusive) for the random integers.
- **shape_tuple**: A tuple defining the shape of the output tensor.

### Example with `torch.randint`:

If you have `batch_size = 4` and you write:

```python
ix = torch.randint(len(data) - block_size, (batch_size,))
```

This code is generating a tensor of shape `(4,)`—meaning it contains four elements. Each element is a random integer between 0 and `len(data) - block_size`. This is particularly useful for selecting starting indices for sequences in data processing, where each index needs to be independently sampled.

### Key Points:

1. **Tuple Notation**: `(batch_size,)` ensures that the argument is passed as a tuple, which is often required by PyTorch functions for defining dimensions.
2. **Usage in PyTorch**: Many PyTorch functions that generate or manipulate tensors require dimensions to be specified as tuples to correctly interpret the intended shape of the tensor.

This method is widely used in data manipulation and batching processes in machine learning workflows to ensure that tensor operations are applied over the correct dimensions.

## Back to Coding

In [12]:
torch.manual_seed(1337)
batch_size = 4 # Number of independent sequences to process simultaneously
block_size = 8 # Length of each sequence or the context used for predicting the next item

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data # determines which dataset to use
    ix = torch.randint(len(data) - block_size, (batch_size,)) # ix: Randomly selects starting indices for sequences within data, avoiding the last block_size
                                                              # characters to prevent index overflow.
                                                                  # (batch_size,) in the end: explanation found above. It's because it needs a shape_tuple to determine its shape :)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension // iterates over each independent sequences in the batch
    for t in range(block_size): # time dimension // iterates over each element within each independent sequence
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [13]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


## Note Taking

An **embedding** in the context of machine learning, specifically in natural language processing (NLP), is a representation of data where similar data points are close to each other in a high-dimensional space. More simply, it's a way of converting categorical data, like words or tokens, into vectors of real numbers which are easier to process while retaining semantic meaning.

### Key Points about Embeddings:

- **Dimensionality Reduction**: While the vocabulary of a language can be vast, embeddings allow each word or token to be represented by a much smaller vector. This is more efficient than one-hot encoding, where the vector length equals the number of words in the vocabulary, with a single '1' and the rest '0's.

- **Semantic Information**: Embeddings capture more than just the identity of words. They can also capture semantic relationships between words. For example, synonyms might be positioned closer together in the embedding space.

- **Training**: Embeddings can be pre-trained on large text corpora using models like Word2Vec, GloVe, or FastText, or they can be trained from scratch along with the neural network's other parameters tailored to a specific task.

### Specifics from Your Code:

In your provided code snippet, the embedding layer (`self.token_embedding_table`) maps each token (represented as an integer) to a vector of size equal to the vocabulary size. Each position in the vector represents a potential feature or characteristic of the token, learned during training. Although using an embedding size equal to the vocabulary size is unusual (as it can lead to high dimensionality and overfitting), it might be useful in certain contexts or for experimental purposes.

This embedding process is crucial for models to effectively learn and make predictions from textual data, allowing them to handle the vast variability of language with a more manageable and meaningful representation.

Query:
so in my case im creating a lookup table where each token point to its respective vector in high dimensional space (which is also called embedding?)?

and what does nn.Embedding(vocab_size, vocab_size) do? what does the input parameter changes?

---

Yes, you've got it right! In your case, the embedding layer acts as a lookup table where each token (represented as an integer index) is mapped to its respective vector in a high-dimensional space. This vector space, where each vector represents a token, is indeed referred to as an "embedding."

### Breaking Down `nn.Embedding(vocab_size, vocab_size)`

The `nn.Embedding` layer in PyTorch is specifically designed to handle such mappings. When you instantiate an `nn.Embedding` with parameters like `(vocab_size, vocab_size)`, you're setting up:

1. **Number of Embeddings (`num_embeddings`)**: The first `vocab_size` parameter specifies the number of distinct tokens that the embedding layer can handle. This is essentially the size of your vocabulary.

2. **Dimension of Each Embedding Vector (`embedding_dim`)**: The second `vocab_size` parameter sets the size of each embedding vector. In typical use cases, this parameter is much smaller than the vocabulary size because embedding vectors are meant to be a dense, information-rich representation of the tokens.

### Function of `nn.Embedding(vocab_size, vocab_size)`
By using `vocab_size` for both parameters, you are creating an embedding layer where:
- There are as many embedding vectors as there are tokens in your vocabulary.
- Each embedding vector has a dimensionality equal to the number of tokens in your vocabulary.

### Practical Implications
- **Dimensionality**: Normally, the dimension of the embedding vectors (`embedding_dim`) is chosen to be much smaller than the vocabulary size to ensure that the embeddings capture the most important features of the data without being redundantly large. A large embedding dimension (especially equal to the vocabulary size) can lead to overfitting where the model learns the training data too well, including noise and details that do not generalize well to new, unseen data.
- **Efficiency**: Large embeddings consume more memory and computational resources, which can make training slower and less efficient.

### Example Use
A common setup might be something like `nn.Embedding(10000, 300)`, where 10,000 represents a vocabulary of 10,000 words, and each word is represented by a 300-dimensional vector. This setup efficiently compresses information while retaining enough complexity to model linguistic nuances.

In your specific case, using `vocab_size` for both might be more experimental or illustrative, helping to explore or demonstrate certain concepts, but it's generally advisable to use a smaller dimension for the embeddings in practical applications.

Query: why do i need to zero_grad first? why not just calculate gradient right away

---

In PyTorch, when you use `.backward()` to calculate gradients during the training of a neural network, the gradients for each parameter are accumulated by default. This means that each time `.backward()` is executed, gradients from the current pass are added to those already stored from previous backward passes, unless they are explicitly cleared.

### Why Zeroing Gradients is Necessary
1. **Accumulation by Default**: By design, PyTorch accumulates gradients because this feature supports several advanced optimization tasks, such as training parts of a model (like in transfer learning) or accumulating gradients over multiple batches or sub-batches. However, in standard training loops, you typically want to update weights based on the gradients computed from just the current batch, not a combination of the current and previous batches.

2. **Correct Gradient Values**: If you don't reset the gradients, the gradient computations would mix gradients from different batches. This would lead to incorrect updates in the weights, potentially causing the model to converge poorly or diverge.

3. **Isolation of Gradient Computation**: Zeroing out the gradients ensures that each batch's gradient computation is independent of others. Each training step's update should reflect only the current batch's data, ensuring that the learning process accurately represents the model's performance on the latest data.

### Usage of `optimizer.zero_grad(set_to_none=True)`
- **Efficiency**: Setting gradients to `None` (`set_to_none=True`) rather than explicitly zeroing them can be more efficient in some cases. When you set gradients to `None`, PyTorch understands that the gradients need to be recalculated afresh, avoiding the allocation of zero tensors before storing the new gradients. This can reduce some overhead, particularly in terms of memory operations, because you skip the explicit zeroing step and go straight to calculating new gradients.

In summary, zeroing out the gradients (or setting them to `None`) before calculating them afresh each time ensures that the updates you make to the model parameters reflect only the most recent data's influence, maintaining the integrity and independence of each training step. This is crucial for the model to learn effectively.

## Back to Coding

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): # inherits from nn.Module <-- PyTorch's base class for all nn modules.

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # first parameter specifies the number of distinct tokens that the embedding layer can handle.
                                                                          # or essentially the size of your vocabulary
                                                                          # second parameter specifies the size of each embedding vector. This is normally much smaller than vocab size itself
                                                                          # because embedding vectors are meant to be a dense, information-rich representation of tokens

    def forward(self, idx, targets=None): # method is part of the model's forward pass. Used for traiing + inference to compute the outputs/logits. And optionally loss if targets are provided

        # idx and targets are both (B,T) tensor of integers // B is batch size, and  is sequence length
        # targets = optional tensor of same shape as 'idx' containing target indices for training
        logits = self.token_embedding_table(idx) # (B,T,C) // C is the embedding dimension

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # uses torch.view to merge batch and sequence length dimensions // preparing it for loss compuation
            targets = targets.view(B*T) # simillarly reshapes for loss compuation
            loss = F.cross_entropy(logits, targets) # computes cross-entropy loss between logits and training. Used for training the model.

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context // idx is initial sequence of indices to start generating from // max_new_tokens = number of tokens to generate
        for _ in range(max_new_tokens): # repeats process for max_new_tokens number of iterations
            # get the predictions
            logits, loss = self(idx) # model's 'forward' method is called with current indices idx as input
            # focus only on the last time step (last layer?)
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution // this line samples from probabilitiy distribution "probs"
                  # torch.multinomial treats each row in probs as a separate distributon and samples num_samples=1 index from each distribution
                  # based on probabilities distribution from softmax // multinomial basically just picks options based on how likely they are to be picked
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) # concatenated.
        return idx

m = BigramLanguageModel(vocab_size) # creates instance of BigramLanguageModel with specified vocabulary size. Which determines how many different tokens the model can handle.
logits, loss = m(xb, yb) # runs a forward pass of model. Input batch + target batch inputted.
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())) # calls the 'generate' method of the model to produce a sequence of new
                                                                                                       # tokens. This method starts with an initial token. In this case, torch.zeros
                                                                                                       # or just 0s, which typically corresponds to a "start" token or padding token.

                                                                                                       # idx = torch.zeros specifies starting token

                                                                                                       # [0].tolist(): The 'generate' method returns a tensor, where the 1st dimension
                                                                                                       # is batch size (1 in this case). The [0] selects the first sequence form batch.
                                                                                                       # And toList() converts the tensor of token indices to a list of integers.

                                                                                                       # decode() is defined previously.

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [16]:
batch_size = 32 # sets the size of each batch to 32. But variable isn't used here.
for steps in range(100): # increase number of steps for good results... // below is training loop.

    # sample a batch of data
    xb, yb = get_batch('train') # retrieves a batch of training data (previously defined function)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) # resets gradients of all model parameter to 0 before backpropagation. "set_to_none=True" arg optimises this process by setting gradients to
                                          # 'None' instead of zeroing them. Which can be more efficient
    loss.backward() # performs backpropagation to compute gradient of loss with respect to model parameters
    optimizer.step() # updates the model's parameters using the computed gradients.

print(loss.item()) # .item() converts a single-value tensor to a Python number.

4.587916374206543


In [17]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())) # regenerating again. 500 tokens this time instead. way better!


xiKi-RJ:CgqVuUa!U?qMH.uk!sCuMXvv!CJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzxlyh'Vl:zyxjKXgC?
lv'QKFiBeviNxO'm!Upm$srm&TqViqiBD3HBP!juEOpmZJyF$Fwfy!PlvWPFC
&WDdP!Ko,px
x
tREOE;AJ.BeXkylOVD3KHp$e?nD,.SFbWWI'ubcL!q-tU;aXmJ&uGXHxJXI&Z!gHRpajj;l.
pTErIBjx;JKIgoCnLGXrJSP!AU-AcbczR?


# Mathematical trick in self-attention